In [1]:
# Emissions Calculation by Vehicle Type


In [261]:
import pandas as pd

In [356]:
def grams_to_tons(value):
	""" Convert grams to tons."""

	value = value/453.592
	value = value/2000

	return value

In [370]:
def calculate_tons(df):
    df.rename(columns={'geog_name':'county', 'avgspeedbinId': 'avgSpeedBinID', 'roadtypeId': 'roadTypeID', 'hourId': 'hourID'},
              inplace=True)

    # Calculate total VMT by vehicle group
    df['light'] = df['sov_vmt']+df['hov2_vmt']+df['hov3_vmt']
    df['medium'] = df['medium_truck_vmt']
    df['heavy'] = df['heavy_truck_vmt']
    # What about buses??
    df.drop(['sov_vmt','hov2_vmt','hov3_vmt','medium_truck_vmt','heavy_truck_vmt','bus_vmt'], inplace=True, axis=1)

    # Melt to pivot vmt by vehicle type columns as rows
    df = pd.melt(df, id_vars=['avgSpeedBinID','roadTypeID','hourID','county'], var_name='veh_type', value_name='vmt')

    newdf = pd.merge(df, df_rates, on=['avgSpeedBinID','roadTypeID','hourID','county','veh_type'], how='left', left_index=False)
    # Calculate total grams of emission 
    newdf['grams_tot'] = newdf['grams_per_mile']*newdf['vmt']
    newdf['tons_tot'] = grams_to_tons(newdf['grams_tot'])
    
    return newdf

In [371]:
# Load rate files
df_rates = pd.DataFrame()
for county in ['King','Kitsap','Pierce','Snohomish']:
#     df1 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_light.csv')
    df1 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_light.csv')
    df1['veh_type'] = 'light'
    df1['county'] = county
#     df2 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_med.csv')
    df2 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_medium.csv')
    df2['veh_type'] = 'medium'
    df2['county'] = county
#     df3 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_heavy.csv')
    df3 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_heavy.csv')
    df3['veh_type'] = 'heavy'
    df3['county'] = county
    
    df_rates = df_rates.append(df1)
    df_rates = df_rates.append(df2)
    df_rates = df_rates.append(df3)

df_rates.rename(columns={'sum(ratePerDistance)': 'grams_per_mile'}, inplace=True)
# df_rates.rename(columns={'avgspeedbinId': 'avgSpeedBinID', 'roadtypeId': 'roadTypeID', 'hourId': 'hourID',
#                         'gramsPerMile': 'grams_per_mile'}, inplace=True)

# Interzonal Trips

In [415]:
df_inter = pd.read_csv(r'L:\vision2050\soundcast\integrated\final_runs\tod\tod_run_8.run_2018_10_29_15_01\2050\outputs\emissions\interzonal_vmt_grouped.csv')
df_inter = calculate_tons(df_inter)

# Intrazonal Trips

In [498]:
df_intra = pd.read_csv(r'L:\vision2050\soundcast\integrated\final_runs\tod\tod_run_8.run_2018_10_29_15_01\2050\outputs\emissions\intrazonal_vmt_grouped.csv')
df_intra.rename(columns={'vehicle_type':'veh_type', 'VMT': 'vmt', 'hourId': 'hourID', 'geog_name': 'county'},inplace=True)
df_intra.drop('tod', axis=1, inplace=True)

df_intra_light = df_intra[df_intra['veh_type'].isin(['sov','hov2','hov3'])]
df_intra_light = df_intra_light.groupby(['county','hourID']).sum()[['vmt']].reset_index()
df_intra_light['veh_type'] = 'light'

df_intra_medium = df_intra[df_intra['veh_type'] == 'mediumtruck']
df_intra_medium['veh_type'] = 'medium'
df_intra_heavy = df_intra[df_intra['veh_type'] == 'heavytruck']
df_intra_heavy['veh_type'] = 'heavy'

df_intra = df_intra_light.append(df_intra_medium)
df_intra = df_intra.append(df_intra_heavy)

# For intrazonals, assume standard speed bin and roadway type for all intrazonal trips
speedbin = 4
roadtype = 5

iz_rates = df_rates[(df_rates['avgSpeedBinID'] == speedbin) &
                    (df_rates['roadTypeID'] == roadtype)]

df_intra = pd.merge(df_intra, iz_rates, on=['hourID','county','veh_type'], how='left', left_index=False)
# Calculate total grams of emission 
df_intra['grams_tot'] = df_intra['grams_per_mile']*df_intra['vmt']
df_intra['tons_tot'] = grams_to_tons(df_intra['grams_tot'])

In [501]:
df_intra[df_intra['pollutantID'] == 98]['tons_tot'].sum()

153.3029325439923

# Starts